In [1]:
import requests, gzip, datetime, io, csv
from collections import defaultdict

In [2]:
dates = []
date1 = '2018-07-14'
date2 = '2018-12-13'
start = datetime.datetime.strptime(date1, '%Y-%m-%d')
end = datetime.datetime.strptime(date2, '%Y-%m-%d')
step = datetime.timedelta(days=1)
while start <= end:
    dates.append(start.strftime("%Y%m%d"))
    start += step

In [ ]:
for date in dates:
    print(date + " (position " + str(dates.index(date)) + ")")
    
    daily_data = defaultdict(int)
    for hour in range(0,24):
        filename = "pageviews-"+date+"-"+str(hour).zfill(2)+"0000.gz"
        print(filename+ ":", end =" ")
        r = requests.get('https://dumps.wikimedia.org/other/pageviews/2018/2018-' + date[4:6] + "/" + filename)
        print("Downloaded", end=" ")
        hourly_file = gzip.open(io.BytesIO(r.content), "rt", encoding="utf8").readlines()
        print("Decompressed", end=" ")
        for line in hourly_file:
            if line.startswith("cs ") or line.startswith("cs.m "):
                daily_data[line.strip().split()[1].replace("_", " ")] += int(line.strip().split()[-2])
        print("Processed")
        
    daily_result = list([entry, daily_data[entry]] for entry in daily_data)
    daily_result.sort(key=lambda x: x[1], reverse=True)
    
    with open(date+".csv","w",encoding="utf8",newline='') as daily_file:
        writer = csv.writer(daily_file)
        writer.writerow(["name", "views"])
        for entry in daily_result:
            writer.writerow(entry)
        
    print(date + " done")